Tentative de code pour enlever la déformation d'une lentille connue, très situationnel, généralement un réglage à la main s'avèrera plus efficace

In [ ]:
import numpy as np
import cv2 as cv
import glob
import rawpy
import re

# Termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

# Définir le numéro constant n
n = 15  # Remplacez par le numéro souhaité

# Récupérer toutes les images correspondant à A011_...
all_images = glob.glob('./images/calibration/*.dng')
images = [img for img in all_images if re.search(rf'A{n:03d}_', img)]

# Initialiser la variable pour suivre le plus faible mean_error
best_mean_error = float('inf')
best_image = None
best_ret, best_mtx, best_dist, best_rvecs, best_tvecs = None, None, None, None, None
i=0
img2= cv.imread('./images/image_deformee_exemple.png')
for fname in images:
    with rawpy.imread(fname) as raw:
        img = raw.postprocess()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, (7, 7), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

            # Calibrate the camera
            ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
            h,  w = img2.shape[:2]
            newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
            print (w,h,roi)
            # undistort
            dst = cv.undistort(img2, mtx, dist, None, newcameramtx)
            cv.imwrite('./output/image_corrigee_'+str(n)+' '+str(i)+'.png', dst)
            i+=1
            # crop the image
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
            # Calculate the mean error
            mean_error = 0
            for i in range(len(objpoints)):
                imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
                mean_error += error

            mean_error /= len(objpoints)
            print(f"Image: {fname}, Mean Error: {mean_error}")

            # Vérifier si c'est le plus faible mean_error
            if mean_error < best_mean_error:
                best_mean_error = mean_error
                best_image = fname
                best_ret, best_mtx, best_dist, best_rvecs, best_tvecs = ret, mtx, dist, rvecs, tvecs

# Afficher le résultat final
if best_image:
    print(f"La meilleure image est {best_image} avec une erreur moyenne de {best_mean_error}")

else:
    print("Aucune image valide trouvée.")

Code pour modifier l'ensemble d'une vidéo avec la déformation estimée par le programme au dessus

In [ ]:
import numpy as np
import cv2 as cv

# Charger la vidéo
video_path = './videos/video_test_deformee.mp4'  # Remplacez par le chemin de votre vidéo
cap = cv.VideoCapture(video_path)

# Vérifier si la vidéo a été chargée correctement
if not cap.isOpened():
    print("Erreur : Impossible de charger la vidéo.")
    exit()

# Obtenir les dimensions de la vidéo
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

# Calculer la nouvelle matrice de caméra pour la correction 
newcameramtx, roi = cv.getOptimalNewCameraMatrix(best_mtx, best_dist, (frame_width, frame_height), 1, (frame_width, frame_height))

# Initialiser l'écriture de la vidéo corrigée
output_path = './output/undistorted_video_'+str(n)+'.mp4'  # Chemin de sortie
fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Codec pour la vidéo
out = cv.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Lire et undistort chaque frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Appliquer la correction de distorsion
    undistorted_frame = cv.undistort(frame, best_mtx, best_dist, None, newcameramtx)

    # Écrire la frame corrigée dans la vidéo de sortie
    out.write(undistorted_frame)

# Libérer les ressources
cap.release()
out.release()
cv.destroyAllWindows()

Un code pour extraire la première frame des vidéos fixeGauche et l'enregistrer sur owncloud, utile si les vidéos ont été retouchées (notament pour résoudre de la distorsion)

In [ ]:
import cv2 as cv
import os

root_dir = './videos/input/'

for dirpath, dirnames, filenames in os.walk(root_dir):
    for dirname in dirnames:
        subdir_path = os.path.join(dirpath, dirname)
        for dirpath2, dirnames2, filenames2 in os.walk(subdir_path):
            for file in filenames2:
                if file.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):  # Prend tous les formats vidéo courants
                    video_path = os.path.join(dirpath2, file)
                    print(f"Traitement de la vidéo : {video_path}")

                    cap = cv.VideoCapture(video_path)
                    if not cap.isOpened():
                        print(f"Erreur : impossible d'ouvrir la vidéo {video_path}")
                        continue
                    if not os.access(video_path, os.R_OK):
                        print("Le fichier n'est pas lisible par ce script.")
                        continue

                    ret, frame = cap.read()
                    cap.release()

                    if ret:
                        image_name = os.path.splitext(file)[0] + '.jpg'
                        output_image_path = os.path.join(dirpath2, image_name)
                        cv.imwrite(output_image_path, frame)
                        print(f"Image sauvegardée : {output_image_path}")
                    else:
                        print(f"Erreur : impossible de lire la première frame de {video_path}")

Un code pour charger une image avant d'y appliquer manuellement une distorsion (un peu de lag), utile pour corriger une distorsion de caméra lors de l'enregistrement d'une vidéo

In [ ]:
import cv2
import numpy as np


# Charger et redimensionner l'image
img = cv2.imread('./images/image_deformee_exemple.png')  # Remplacez par le chemin de votre image
img = cv2.resize(img, (1024, 540))

h, w = img.shape[:2]
cx, cy = w // 2, h // 2  # Centre de l'image

# Variables globales
clicked_points = []
k_value = -0.1
distorted_img = None


def distort_image(img, k):
    x, y = np.meshgrid(np.arange(w), np.arange(h))
    x_norm = (x - cx) / cx
    y_norm = (y - cy) / cy
    r = np.sqrt(x_norm**2 + y_norm**2)
    r_new = r + k * (r**3)
    r_nonzero = np.where(r == 0, 1, r)
    x_new = cx + (x_norm / r_nonzero) * r_new * cx
    y_new = cy + (y_norm / r_nonzero) * r_new * cy
    return cv2.remap(img, x_new.astype(np.float32), y_new.astype(np.float32), interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)


def distort_points(points, cx, cy, k):
    points = np.array(points, dtype=np.float32)
    x = points[:, 0]
    y = points[:, 1]
    x_norm = (x - cx) / cx
    y_norm = (y - cy) / cy
    r = np.sqrt(x_norm**2 + y_norm**2)
    r_new = r + k * (r**3)
    r_nonzero = np.where(r == 0, 1, r)
    x_new = cx + (x_norm / r_nonzero) * r_new * cx
    y_new = cy + (y_norm / r_nonzero) * r_new * cy
    return list(zip(x_new, y_new))


def update_display():
    global distorted_img
    distorted_img = distort_image(img, k_value)
    display_img = distorted_img.copy()

    if len(clicked_points) == 2:
        distorted_pts = distort_points(clicked_points, cx, cy, k_value)
        pt1 = tuple(map(int, distorted_pts[0]))
        pt2 = tuple(map(int, distorted_pts[1]))
        cv2.line(display_img, pt1, pt2, (255, 0, 0), 2)
        for pt in [pt1, pt2]:
            cv2.circle(display_img, pt, 5, (0, 255, 0), -1)

    cv2.imshow('Effet lentille', display_img)


def on_mouse(event, x, y, flags, param):
    global clicked_points
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(clicked_points) < 2:
            clicked_points.append((x, y))
        else:
            clicked_points = [(x, y)]
        update_display()


def on_trackbar(val):
    global k_value
    k_value = (val - 100) / 1000.0  # de -0.1 à 0.1
    update_display()


# Création fenêtre
cv2.namedWindow('Effet lentille', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Effet lentille', 1024, 540)
cv2.setMouseCallback('Effet lentille', on_mouse)
cv2.createTrackbar('k', 'Effet lentille', 90, 200, on_trackbar)

update_display()
cv2.waitKey(0)
cv2.destroyAllWindows()

Un code pour appliquer cette déformation à l'ensemble de la vidéo

In [ ]:
import cv2
import numpy as np
import os





def process_video(input_path, output_path, k, target_width=1024):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Erreur : impossible d'ouvrir la vidéo {input_path}")
        return

    # Lire les propriétés
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Redimensionnement pour correspondre à ce que l'utilisateur a vu (par défaut 1024x540)
    scale = target_width / original_width
    target_height = int(original_height * scale)
    cx, cy = target_width // 2, target_height // 2

    # Préparer le writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

    # Pré-calculer la distorsion (meshgrid)
    x, y = np.meshgrid(np.arange(target_width), np.arange(target_height))
    x_norm = (x - cx) / cx
    y_norm = (y - cy) / cy
    r = np.sqrt(x_norm**2 + y_norm**2)
    r_new = r + k * (r**3)
    r_nonzero = np.where(r == 0, 1, r)
    map_x = cx + (x_norm / r_nonzero) * r_new * cx
    map_y = cy + (y_norm / r_nonzero) * r_new * cy
    map_x = map_x.astype(np.float32)
    map_y = map_y.astype(np.float32)

    print("Début du traitement de la vidéo...")

    frame_index = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionner
        frame = cv2.resize(frame, (target_width, target_height))

        # Appliquer la distorsion
        distorted_frame = cv2.remap(frame, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        out.write(distorted_frame)

        frame_index += 1
        if frame_index % 50 == 0:
            print(f"Frame {frame_index}/{frame_count} traitée...")

    cap.release()
    out.release()
    print(f" Vidéo enregistrée à : {output_path}")



# Appliquer à une vidéo
input_video = './videos/input.mp4'  # Mets ici ton chemin d'entrée
output_video = './videos/output_distorted.mp4'  # Chemin de sortie
process_video(input_video, output_video, k_value)